# Pokemon Match

This file calculates the top x number of attacking pokemon per type for raids (PvE). It lists:
- the top moves for each type
- a summary of the pokemon for each type (split by normal/shadow) 
- a combined summary of all pokemon of all types at the end (split norm/shadow)

Use the "Runtime -> Run All" to see all output :)

In [1]:
import os
import shutil
print(os.getcwd())

/content


In [2]:
# this bit imports the github, clones and changes the path:
os.chdir('/content')
myDir = './drmckenzie.github.io/'
 
# WARNING! this will delete any changes to the files you have made
if os.path.isdir(myDir):
    shutil.rmtree(myDir)
 
!git clone https://github.com/drmckenzie/drmckenzie.github.io.git
 
os.chdir(myDir)
print(os.getcwd())

Cloning into 'drmckenzie.github.io'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 296 (delta 197), reused 212 (delta 122), pack-reused 0
Receiving objects: 100% (296/296), 1.47 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (197/197), done.
/content/drmckenzie.github.io


# You can change these default settings to customise the results:

In [3]:
# This controls the score filter:
pvpFilter = 80

# This controls the filter taking the top however many of the leage results, e.g. top 100:
pvpTopX = 100

In [4]:
import pandas as pd
 
import pokemonMatch as match
 
# PvPpoke filename
greatFilename = 'data/great_overall.csv'
ultraFilename = 'data/ultra_overall.csv'
masterFilename = 'data/master_overall.csv'
retroFilename = 'data/retro_overall.csv'

remixFilename = 'data/great_remix_overall.csv'
classicFilename = 'data/master_classic_overall.csv'

# setting the filters
typeFilter = []
topX = 0

# load data
greatData=match.getPvpPokeData(greatFilename,typeFilter,topX)
ultraData=match.getPvpPokeData(ultraFilename,typeFilter,topX)
masterData=match.getPvpPokeData(masterFilename,typeFilter,topX)
retroData=match.getPvpPokeData(retroFilename,typeFilter,topX)

remixData=match.getPvpPokeData(remixFilename,typeFilter,topX)
classicData=match.getPvpPokeData(classicFilename,typeFilter,topX)


In [5]:
#reduce data
columnList=["Pokemon","#","Score"]

greatData = greatData[columnList]
ultraData = ultraData[columnList]
masterData = masterData[columnList]
retroData = retroData[columnList]

remixData = remixData[columnList]
classicData = classicData[columnList]
 
greatData.rename(columns={"Score": "GL"},inplace=True)
ultraData.rename(columns={"Score": "UL"},inplace=True)
masterData.rename(columns={"Score": "ML"},inplace=True)
retroData.rename(columns={"Score": "RL"},inplace=True)
 
remixData.rename(columns={"Score": "XG"},inplace=True)
classicData.rename(columns={"Score": "CL"},inplace=True)
 
# add index for all dataframes
greatData = match.addNewIndexCol(greatData,'GLx')
ultraData = match.addNewIndexCol(ultraData,'ULx')
masterData = match.addNewIndexCol(masterData,'MLx')
retroData = match.addNewIndexCol(retroData,'RLx')

remixData = match.addNewIndexCol(remixData,'XGx')
classicData = match.addNewIndexCol(classicData,'CLx')

classicData.head(10)

,Pokemon,#,CL,CLx
0,Groudon,383.0,94.9,1
1,Melmetal,809.0,92.6,2
3,Snorlax (Shadow),143.0,90.5,4
2,Giratina (Origin),487.0,90.5,3
4,Dialga,483.0,90.0,5
7,Mewtwo,150.0,89.2,8
6,Ho-Oh,250.0,89.2,7
5,Dragonite (Shadow),149.0,89.2,6
9,Snorlax,143.0,88.8,10
8,Palkia,484.0,88.8,9


In [6]:
setData = pd.DataFrame()
 
#merge
mergeList=["Pokemon","#"]
setData=greatData.merge(ultraData, left_on=mergeList, right_on=mergeList, how="outer")
setData=setData.merge(masterData, left_on=mergeList, right_on=mergeList, how="outer")
setData=setData.merge(retroData, left_on=mergeList, right_on=mergeList, how="outer")

setData=setData.merge(remixData, left_on=mergeList, right_on=mergeList, how="outer")
setData=setData.merge(classicData, left_on=mergeList, right_on=mergeList, how="outer")

# control the type:
setData.astype({'#': 'int32'}).dtypes

#reorder column order:
#setData = setData.reindex(columns=['Pokemon','#','GL','GLx','UL','ULx','ML','MLx','RL','RLx'])
setData = setData.reindex(columns=['Pokemon','#','GL','GLx','XG','XGx','RL','RLx','UL','ULx','ML','MLx','CL','CLx'])
#setData = setData.reindex(columns=['Pokemon','#','GL','UL','ML','RL','GLx','ULx','MLx','RLx'])
setData = setData.reindex(columns=['Pokemon','#','GL','XG','RL','UL','ML','CL','GLx','XGx','RLx','ULx','MLx','CLx'])

# control what gets shown:
StP = str(pvpFilter)
dispData  = setData.query('GL>='+StP+' or XG>='+StP+' or RL>='+StP+' or UL>='+StP+' or ML>='+StP+' or CL>='+StP )
trashData = setData.query('GL<'+StP+' and XG<'+StP+' and RL<'+StP+' and UL<'+StP+' and ML<'+StP+' and CL<'+StP  )
 
#setData=setData.merge(ultraData, left_on=mergeList, right_on=mergeList,suffixes=('', '_UL'))

# make the NaN a big value:
setData.loc[setData['GLx'].isna(),"GLx"] = 9999
setData.loc[setData['ULx'].isna(),"ULx"] = 9999
setData.loc[setData['MLx'].isna(),"MLx"] = 9999
setData.loc[setData['RLx'].isna(),"RLx"] = 9999

setData.loc[setData['XGx'].isna(),"XGx"] = 9999
setData.loc[setData['CLx'].isna(),"CLx"] = 9999

setData.tail(100)

,Pokemon,#,GL,XG,RL,UL,ML,CL,GLx,XGx,RLx,ULx,MLx,CLx
570,Espurr,677.0,56.8,58.2,58.7,NaN,NaN,NaN,571.0,552.0,459.0,9999.0,9999.0,9999.0
571,Gorebyss,368.0,56.6,56.8,52.1,54.9,NaN,NaN,572.0,564.0,501.0,373.0,9999.0,9999.0
572,Geodude,74.0,56.5,58.9,62.1,NaN,NaN,NaN,573.0,547.0,420.0,9999.0,9999.0,9999.0
573,Chatot,441.0,56.4,57.4,56.7,NaN,NaN,NaN,574.0,557.0,479.0,9999.0,9999.0,9999.0
574,Cofagrigus,563.0,56.0,57.0,58.8,50.5,30.1,30.0,575.0,561.0,457.0,387.0,305.0,297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,Darkrai,491.0,NaN,NaN,NaN,62.0,69.7,70.8,9999.0,9999.0,9999.0,334.0,72.0,78.0
666,Terrakion,639.0,NaN,NaN,NaN,60.0,63.0,64.2,9999.0,9999.0,9999.0,348.0,126.0,127.0
667,Tornadus (Incarnate),641.0,NaN,NaN,NaN,57.5,48.6,46.5,9999.0,9999.0,9999.0,363.0,239.0,257.0
668,Regigigas,486.0,NaN,NaN,NaN,42.1,63.0,68.2,9999.0,9999.0,9999.0,399.0,125.0,96.0


In [7]:
print("The score filter is set at : "+str(pvpFilter))
print("")

# combine trash data mon:
keepList = [int(i) for i in dispData['#'] ]
keepList = list(sorted(set(keepList)))
keepListString = match.concatenateListOfMon(keepList)
print("This is the list to KEEP pokemon for PvP - any above score of "+str(pvpFilter))
print(keepListString)

print("")

# combine trash data mon:
trashList = [int(i) for i in trashData['#'] ]
trashList = list(sorted(set(trashList)))
trashListString = match.concatenateListOfMon(trashList)
print("This is the list to TRASH pokemon not for PvP - ALL below score of "+str(pvpFilter))
print(trashListString)

The score filter is set at : 80

This is the list to KEEP pokemon for PvP - any above score of 80
2-3,6,9,15,26,28,36,38,40,42,57,62,67-68,75,78,82,87,89,93-94,97,105,107-108,126-127,130-131,134,143-145,148-151,154,157,164,171,182,184-186,189,195,197,202,205,207-208,210,212,214,226-227,237,243,245,249-250,254,257,259-260,264,272,275,279,282,288,297,302,308,330,334-335,340,346,350-351,356-357,362,364,376-379,381-383,385-386,395,407,411,421,423,426,430,435,437,445-446,448,454,460,462-466,468,472-473,475-476,478,483-484,487-488,494,497,523,530,547,558,560,565,589,593-594,596,598,618,630,634,640,643-645,649,651-652,660,663,809,862-863,865-865

This is the list to TRASH pokemon not for PvP - ALL below score of 80
28-28,31,34,45,55,59,65,73,76,80,89,91,99,103,106,110,112,115,121,123,128,135-136,139,141-142,146,160,169,181,196,199,217,221,230,232-233,241-242,244,251,289,295,326,337-338,348,365,369,373,380,384,386,389,392,398,405,409,429,450,467,469-471,474,477,479-482,500,503,508,518,521,526,

In [8]:
setDataTopX = pd.DataFrame()
 
#merge
mergeList=["Pokemon","#"]
setDataTopX=greatData.merge(ultraData, left_on=mergeList, right_on=mergeList, how="outer")
setDataTopX=setDataTopX.merge(masterData, left_on=mergeList, right_on=mergeList, how="outer")
setDataTopX=setDataTopX.merge(retroData, left_on=mergeList, right_on=mergeList, how="outer")

setDataTopX=setDataTopX.merge(remixData, left_on=mergeList, right_on=mergeList, how="outer")
setDataTopX=setDataTopX.merge(classicData, left_on=mergeList, right_on=mergeList, how="outer")

# control the type:
setDataTopX.astype({'#': 'int32'}).dtypes

#reorder column order:
setDataTopX = setDataTopX.reindex(columns=['Pokemon','#','GL','GLx','XG','XGx','RL','RLx','UL','ULx','ML','MLx','CL','CLx'])
setDataTopX = setDataTopX.reindex(columns=['Pokemon','#','GL','XG','RL','UL','ML','CL','GLx','XGx','RLx','ULx','MLx','CLx'])

import numpy as np

# rank them by usefulness
setDataTopX['SumRank'] = setDataTopX.loc[:,['GLx','XGx','RLx','ULx','MLx','CLx']].sum(axis=1)
setDataTopX.loc[setDataTopX['SumRank'] == 0] = np.nan
setDataTopX.sort_values(by=['SumRank'],ascending=True,inplace=True)

# make the NaN a big value:
setDataTopX.loc[setDataTopX['GLx'].isna(),"GLx"] = 999
setDataTopX.loc[setDataTopX['ULx'].isna(),"ULx"] = 999
setDataTopX.loc[setDataTopX['MLx'].isna(),"MLx"] = 999
setDataTopX.loc[setDataTopX['RLx'].isna(),"RLx"] = 999

setDataTopX.loc[setDataTopX['XGx'].isna(),"XGx"] = 9999
setDataTopX.loc[setDataTopX['CLx'].isna(),"CLx"] = 9999

setDataTopX.head(100)

# control what gets shown:
StX = str(pvpTopX)
#dispDataTopX = setDataTopX.query('GLx <= 5' )
#trashDataTopX = setDataTopX.query('GLx > 10')+str(pvpTopX) )

dispDataTopX  = setDataTopX.query('GLx<='+StX+' or XGx<='+StX+' or RLx<='+StX+' or ULx<='+StX+' or MLx<='+StX+' or CLx<='+StX )
trashDataTopX = setDataTopX.query('GLx>'+StX+' and XGx>'+StX+' and RLx>'+StX+' and ULx>'+StX+' and MLx>'+StX+' and CLx>'+StX  )

dispDataTopX.head(300)
#trashDataTopX.head(100)


,Pokemon,#,GL,XG,RL,UL,ML,CL,GLx,XGx,RLx,ULx,MLx,CLx,SumRank
7,Azumarill,184.0,90.8,NaN,NaN,NaN,NaN,NaN,8.0,9999.0,999.0,999.0,999.0,9999.0,8.0
645,Giratina (Origin),487.0,NaN,NaN,NaN,88.5,92.2,90.5,999.0,9999.0,999.0,14.0,2.0,3.0,19.0
17,Bastiodon,411.0,89.2,92.6,NaN,NaN,NaN,NaN,19.0,4.0,999.0,999.0,999.0,9999.0,23.0
642,Giratina (Altered),487.0,NaN,NaN,NaN,92.6,87.8,85.2,999.0,9999.0,999.0,4.0,7.0,19.0,30.0
11,Zweilous,634.0,89.9,89.0,NaN,NaN,NaN,NaN,13.0,21.0,999.0,999.0,999.0,9999.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,Rhyperior,464.0,67.2,67.4,68.7,64.0,73.4,83.2,417.0,429.0,309.0,317.0,50.0,28.0,1550.0
394,Braviary,628.0,68.1,68.1,65.9,71.5,63.4,68.1,393.0,407.0,360.0,221.0,120.0,97.0,1598.0
471,Rayquaza,384.0,64.6,65.7,63.3,67.5,68.7,68.4,472.0,455.0,397.0,275.0,80.0,94.0,1773.0
514,Porygon-Z,474.0,61.3,61.1,62.6,65.4,66.6,74.7,515.0,515.0,413.0,301.0,98.0,53.0,1895.0


In [9]:
print("The score cutoff for top number of pokemon in the league is set at top: "+str(pvpTopX))
print("")

# combine trash data mon:
keepList = [int(i) for i in dispDataTopX['#'] ]
keepList = list(sorted(set(keepList)))
keepListString = match.concatenateListOfMon(keepList)
print("This is the list to KEEP pokemon for PvP - any ranked below the top "+str(pvpTopX))
print(keepListString)

print("")

# combine trash data mon:
trashList = [int(i) for i in trashDataTopX['#'] ]
trashList = list(sorted(set(trashList)))

#print(trashList)

trashListString = match.concatenateListOfMon(trashList)
print("This is the list to TRASH pokemon not for PvP - ALL ranked above the top "+str(pvpTopX))
print(trashListString)

The score cutoff for top number of pokemon in the league is set at top: 100

This is the list to KEEP pokemon for PvP - any ranked below the top 100
3-3,6,9,15,26,28,36,38,40,42,57,62,67-68,75-76,87,89,93-94,97,105,107-108,127,130-131,134,143-146,148-151,154,157,164,171,182,184,186,189,195,197,205,208,210,212,214,217,226-227,237,243-245,248-251,254,257,259-260,264,275,279,282,288,297,302,308,330,334-335,340,346,350-351,356-357,362,364,369,376-386,395,411,426,430,445-446,448,450,454,460-468,471-475,478,483-488,491,494,497,530,534,547,553,555,558,560,565,589,593-594,596,598,609,612,614,618,626,628,630,634-635,638,640,642-645,649,652,660,663,809,862-863,865-865

This is the list to TRASH pokemon not for PvP - ALL ranked above the top 100
2-2,5,8,12,17-18,20,22,24,26-28,30-31,33-35,43-45,47,49,51,53,55,58-59,61,64-66,70-71,73-83,85,88-89,91,95-96,98-99,101-106,109-115,117,119,121-128,135-142,153,156,159-160,162,166,168-170,176,178,180-182,185,189-190,192-193,195-196,198-200,202-204,206-207

In [20]:
# which leagues is good for this pokemon?
# using the top x result, I think this is more consistent.
setDataTopX["Best"] = ''
setDataTopX["BestNo"] = 0
for i in range(len(setDataTopX)):
    setDataTopX.loc[i,"Best"] = [['']]
    if setDataTopX.loc[i,"GLx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("GL")
    if setDataTopX.loc[i,"XGx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("XG")
    if setDataTopX.loc[i,"RLx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("RL")
    if setDataTopX.loc[i,"ULx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("UL")
    if setDataTopX.loc[i,"MLx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("ML")
    if setDataTopX.loc[i,"CLx"]<pvpTopX:
        setDataTopX.loc[i,"Best"].append("CL")
    if setDataTopX.loc[i,"Best"]==['']:
        setDataTopX.loc[i,"Best"].append("-")
    # remove the first ''
    setDataTopX.loc[i,"Best"].pop(0)
    setDataTopX.loc[i,"BestNo"] = len(setDataTopX.loc[i,"Best"])


setDataTopX.sort_values(by=['BestNo','SumRank'],ascending=(False,True),inplace=True)

# dump this to CSV file, top directory:
outCsvFilename = '../csv_Top_X_pokemon_by_leagues.csv'
setDataTopX.to_csv(outCsvFilename)

setDataTopX.head(200)

,Pokemon,#,GL,XG,RL,UL,ML,CL,GLx,XGx,RLx,ULx,MLx,CLx,SumRank,Best,BestNo
19,Regirock,377.0,89.1,90.5,92.0,88.8,76.1,82.8,20.0,14.0,5.0,13.0,40.0,29.0,121.0,"[GL, XG, RL, UL, ML, CL]",6
6,Machamp (Shadow),68.0,91.0,92.0,85.3,86.2,79.2,77.3,7.0,6.0,34.0,29.0,28.0,42.0,146.0,"[GL, XG, RL, UL, ML, CL]",6
9,Cresselia,488.0,90.2,93.1,93.8,95.4,71.0,68.4,10.0,3.0,2.0,1.0,64.0,93.0,173.0,"[GL, XG, RL, UL, ML, CL]",6
15,Mew,151.0,89.6,88.5,87.5,83.0,81.3,82.2,17.0,27.0,19.0,62.0,23.0,31.0,179.0,"[GL, XG, RL, UL, ML, CL]",6
55,Snorlax,143.0,85.9,87.1,87.3,86.1,82.1,88.8,56.0,39.0,22.0,32.0,22.0,10.0,181.0,"[GL, XG, RL, UL, ML, CL]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Linoone,264.0,80.7,78.1,83.2,NaN,NaN,NaN,146.0,197.0,57.0,999.0,999.0,9999.0,400.0,[RL],1
222,Mawile,303.0,76.1,78.2,NaN,NaN,NaN,NaN,219.0,195.0,999.0,999.0,999.0,9999.0,414.0,[-],1
211,Dusclops,356.0,76.7,80.1,83.6,NaN,NaN,NaN,211.0,161.0,48.0,999.0,999.0,9999.0,420.0,[RL],1
200,Linoone (Galarian),264.0,77.4,75.8,NaN,NaN,NaN,NaN,200.0,235.0,999.0,999.0,999.0,9999.0,435.0,[-],1
